# Hamilton Best Practices

This notebook demonstrates the Hamilton best practices we've adopted for our data pipeline development. These practices ensure maintainable, scalable, and debuggable dataflows.

## Setup

First, let's load the Hamilton Jupyter magic and import necessary libraries:

In [ ]:
# Load Hamilton Jupyter magic
%load_ext hamilton.plugins.jupyter_magic

import pandas as pd
import geopandas as gpd
import pyarrow as pa
from typing import Dict, List
from hamilton.function_modifiers import config, cache, tag
from hamilton.htypes import Parallelizable, Collect
import ipywidgets as widgets
from IPython.display import display

## Interactive Configuration

Use this dropdown to select execution mode for the configurable examples:

In [ ]:
# Interactive execution mode selector
execution_mode_dropdown = widgets.Dropdown(
    options=['sequential', 'parallel'],
    value='sequential',
    description='Execution Mode:',
    style={'description_width': 'initial'}
)

display(execution_mode_dropdown)

# Function to get current selection
def get_execution_mode():
    return execution_mode_dropdown.value

print(f"Current execution mode: {get_execution_mode()}")

## Core Principles

### 1. Function Naming Conventions

**✅ Use nouns, not verbs for function names**

Hamilton functions represent data artifacts/nouns in your pipeline, not actions/verbs.

**Reference**: [Function Naming Best Practices](https://hamilton.dagworks.io/en/latest/concepts/best-practices/function-naming/)

In [ ]:
%%cell_to_module good_naming --display

import pandas as pd
import geopandas as gpd
import pyarrow as pa
from shapely.geometry import Point

# ✅ Good - noun-based naming
def raw_data() -> pd.DataFrame:
    """Raw input data."""
    return pd.DataFrame({'x': [1, 2, 3], 'y': [4, 5, 6]})

def processed_geodataframe(raw_data: pd.DataFrame) -> gpd.GeoDataFrame:
    """Processed geodataframe with geometry."""
    gdf = gpd.GeoDataFrame(raw_data)
    gdf['geometry'] = [Point(x, y) for x, y in zip(gdf.x, gdf.y)]
    return gdf

def geoarrow_table(processed_geodataframe: gpd.GeoDataFrame) -> pa.Table:
    """Arrow table with geospatial data."""
    return pa.Table.from_pandas(processed_geodataframe, preserve_index=False)

**❌ Bad - verb-based naming (don't do this)**

Compare the DAG visualization below - verb-based naming makes it harder to understand what data artifacts the pipeline produces:

In [ ]:
%%cell_to_module bad_naming --display

import pandas as pd
import geopandas as gpd
import pyarrow as pa
from shapely.geometry import Point

# ❌ Bad - verb-based naming  
def load_raw_data() -> pd.DataFrame:
    """Function name is a verb - confusing!"""
    return pd.DataFrame({'x': [1, 2, 3], 'y': [4, 5, 6]})

def process_geodataframe(load_raw_data: pd.DataFrame) -> gpd.GeoDataFrame:
    """Another verb - what does this represent?"""
    gdf = gpd.GeoDataFrame(load_raw_data)
    gdf['geometry'] = [Point(x, y) for x, y in zip(gdf.x, gdf.y)]
    return gdf

def convert_to_geoarrow(process_geodataframe: gpd.GeoDataFrame) -> pa.Table:
    """Verb-based naming makes DAG harder to read."""
    return pa.Table.from_pandas(process_geodataframe, preserve_index=False)

### 2. Helper Functions

**✅ Prefix helper functions with underscore**

Helper functions with `_` prefix are excluded from DAG visualization and execution tracking, keeping the pipeline graph clean.

**Reference**: [Code Organization Best Practices](https://hamilton.dagworks.io/en/latest/concepts/best-practices/code-organization/)

In [ ]:
%%cell_to_module helper_example --display

import pandas as pd
import geopandas as gpd
import pyarrow as pa
from shapely.geometry import Point
from typing import Dict

# ✅ Good - helper functions excluded from DAG
def _geoarrow_conversion_helper(gdf: gpd.GeoDataFrame, dataset_name: str) -> pa.Table:
    """Helper function for GeoArrow conversion - won't appear in DAG."""
    # Complex conversion logic here
    table = pa.Table.from_pandas(gdf, preserve_index=False)
    metadata = {"dataset_name": dataset_name}
    return table.replace_schema_metadata(metadata)

def _duckdb_storage_helper(table: pa.Table, conn) -> int:
    """Helper function for DuckDB storage - won't appear in DAG."""
    # Complex storage logic here
    return len(table)

# Hamilton DAG nodes that use helpers
def sample_data() -> gpd.GeoDataFrame:
    """Sample geodataframe."""
    df = pd.DataFrame({'x': [1, 2], 'y': [3, 4], 'name': ['A', 'B']})
    gdf = gpd.GeoDataFrame(df)
    gdf['geometry'] = [Point(x, y) for x, y in zip(gdf.x, gdf.y)]
    return gdf

def processed_arrow_table(sample_data: gpd.GeoDataFrame) -> pa.Table:
    """Processed arrow table using helper function."""
    return _geoarrow_conversion_helper(sample_data, "sample_dataset")

def storage_result(processed_arrow_table: pa.Table) -> Dict:
    """Storage result using helper function."""
    # In real implementation, would pass actual connection
    row_count = len(processed_arrow_table)
    return {"rows_stored": row_count, "status": "success"}

### 3. Dependency Injection Over Direct Calls

**✅ Use Hamilton's dependency injection**

Dependency injection enables Hamilton's caching, parallelization, and visualization features.

**Reference**: [Hamilton Node Concepts](https://hamilton.dagworks.io/en/latest/concepts/node/)

In [ ]:
%%cell_to_module dependency_injection_good --display

import pandas as pd
import geopandas as gpd
import pyarrow as pa
from shapely.geometry import Point
from typing import Dict

# ✅ Good - dependency injection
def config_params() -> Dict:
    """Configuration parameters."""
    return {"buffer_size": 100, "crs": "EPSG:4326"}

def raw_data() -> pd.DataFrame:
    """Raw input data."""
    return pd.DataFrame({'x': [1, 2, 3], 'y': [4, 5, 6], 'value': [10, 20, 30]})

def processed_geodataframe(raw_data: pd.DataFrame, config_params: Dict) -> gpd.GeoDataFrame:
    """Processed geodataframe using injected dependencies."""
    gdf = gpd.GeoDataFrame(raw_data)
    gdf['geometry'] = [Point(x, y) for x, y in zip(gdf.x, gdf.y)]
    gdf = gdf.set_crs(config_params["crs"])
    return gdf

def arrow_table(processed_geodataframe: gpd.GeoDataFrame) -> pa.Table:
    """Arrow table from processed geodataframe."""
    return pa.Table.from_pandas(processed_geodataframe, preserve_index=False)

**❌ Bad - direct function calls (don't do this)**

This breaks Hamilton's dependency tracking and prevents caching/parallelization:

In [ ]:
%%cell_to_module dependency_injection_bad --display

import pandas as pd
import geopandas as gpd
import pyarrow as pa
from shapely.geometry import Point
from typing import Dict

# ❌ Bad - direct function calls
def config_params() -> Dict:
    """Configuration parameters."""
    return {"buffer_size": 100, "crs": "EPSG:4326"}

def raw_data() -> pd.DataFrame:
    """Raw input data."""
    return pd.DataFrame({'x': [1, 2, 3], 'y': [4, 5, 6], 'value': [10, 20, 30]})

def arrow_table_bad() -> pa.Table:
    """Arrow table with direct calls - breaks Hamilton's dependency tracking!"""
    # Direct function calls - Hamilton can't track these dependencies
    data = raw_data()  # Direct call
    config = config_params()  # Direct call
    
    gdf = gpd.GeoDataFrame(data)
    gdf['geometry'] = [Point(x, y) for x, y in zip(gdf.x, gdf.y)]
    gdf = gdf.set_crs(config["crs"])
    
    return pa.Table.from_pandas(gdf, preserve_index=False)

### 4. Configuration and Execution Modes

**✅ Use config.when for conditional execution**

The key fix: Use `Collect[str]` for parallel mode to properly handle the collected results from `Parallelizable[str]`.

**Reference**: [Parallel Task Concepts](https://hamilton.dagworks.io/en/latest/concepts/parallel-task/)

Use the dropdown above to switch between execution modes and see how the DAG changes:

In [ ]:
%%cell_to_module execution_modes --display --config execution_mode=sequential

import pandas as pd
from typing import Dict, List
from hamilton.function_modifiers import config
from hamilton.htypes import Parallelizable, Collect

def metadata() -> Dict:
    """Sample metadata for datasets."""
    return {
        "dataset_a": {"url": "http://example.com/a", "size": 100},
        "dataset_b": {"url": "http://example.com/b", "size": 200},
        "dataset_c": {"url": "http://example.com/c", "size": 150}
    }

@config.when(execution_mode="parallel")
def dataset_names__parallel(metadata: Dict) -> Parallelizable[str]:
    """Dataset names for parallel processing."""
    for name in metadata.keys():
        yield name

@config.when(execution_mode="sequential") 
def dataset_names__sequential(metadata: Dict) -> List[str]:
    """Dataset names for sequential processing."""
    return list(metadata.keys())

# Fix: Use Collect for parallel mode, direct List for sequential
@config.when(execution_mode="parallel")
def processing_summary__parallel(dataset_names: Collect[str], metadata: Dict) -> Dict:
    """Summary of datasets to process (parallel mode)."""
    dataset_list = list(dataset_names)  # Convert Collect to list
    total_size = sum(metadata[name]["size"] for name in dataset_list)
    return {
        "total_datasets": len(dataset_list),
        "total_size": total_size,
        "datasets": dataset_list
    }

@config.when(execution_mode="sequential")
def processing_summary__sequential(dataset_names: List[str], metadata: Dict) -> Dict:
    """Summary of datasets to process (sequential mode)."""
    total_size = sum(metadata[name]["size"] for name in dataset_names)
    return {
        "total_datasets": len(dataset_names),
        "total_size": total_size,
        "datasets": dataset_names
    }

### 5. Modular Organization

**✅ Keep dataflow files under 500 lines**

Break large dataflows into focused modules for maintainability.

**Reference**: [Code Organization](https://hamilton.dagworks.io/en/latest/concepts/best-practices/code-organization/)

In [ ]:
%%cell_to_module modular_organization --display

import pandas as pd
import geopandas as gpd
from typing import Dict

# ✅ Good - focused module for data loading
def raw_dataset_metadata() -> Dict:
    """Metadata for raw datasets - focused on data loading."""
    return {
        "pv_locations": {"source": "doi:10.1234/pv-data", "format": "geojson"},
        "admin_boundaries": {"source": "overture", "format": "parquet"},
        "irradiance_data": {"source": "nrel", "format": "zarr"}
    }

def dataset_priorities(raw_dataset_metadata: Dict) -> Dict:
    """Processing priorities for datasets."""
    return {
        "pv_locations": 1,  # High priority
        "admin_boundaries": 2,  # Medium priority  
        "irradiance_data": 3   # Low priority
    }

def processing_plan(raw_dataset_metadata: Dict, dataset_priorities: Dict) -> Dict:
    """Combined processing plan."""
    return {
        "datasets": list(raw_dataset_metadata.keys()),
        "total_datasets": len(raw_dataset_metadata),
        "high_priority": [k for k, v in dataset_priorities.items() if v == 1]
    }

**Example module structure for our project:**

```
dataflows/
├── data_loading.py      # DOI downloads, STAC queries
├── geospatial.py        # GeoDataFrame processing, CRS handling
├── geoarrow_conversion.py  # Arrow table creation, DuckDB export
├── validation.py        # Data quality checks
└── visualization.py     # DAG visualization helpers
```

### 6. Strategic Caching

**✅ Cache expensive operations, avoid problematic data types**

**Reference**: [Hamilton Caching](https://hamilton.dagworks.io/en/latest/concepts/caching/)

In [ ]:
%%cell_to_module caching_example --display

# Cache expensive downloads
@cache(behavior="default", format="json")
def dataset_download_path() -> str:
    """Expensive download operation - cached."""
    # Simulate expensive download
    import time
    time.sleep(0.1)  # Simulate network delay
    return "/tmp/downloaded_dataset.json"

# Disable caching for problematic data types
@cache(behavior="disable")  # GeoArrow extension types have serialization issues
def geoarrow_table() -> pa.Table:
    """GeoArrow table - caching disabled due to serialization issues."""
    # Simulate GeoArrow table creation
    return pa.table({"geometry": [b"\x01\x01\x00\x00\x00"], "id": [1]})

def final_result(dataset_download_path: str, geoarrow_table: pa.Table) -> Dict:
    """Final processing result."""
    return {
        "source_path": dataset_download_path,
        "table_rows": len(geoarrow_table),
        "status": "processed"
    }

## Summary

### Key Fixes Applied:

1. **Correct Magic Syntax**: Used `%%cell_to_module` instead of `%%hamilton_module`
2. **Fixed Type Compatibility**: Used `Collect[str]` for parallel mode to properly handle collected results from `Parallelizable[str]`
3. **Proper Function Naming**: All functions use noun-based naming with helper functions prefixed with `_`
4. **Strategic Caching**: Demonstrated proper caching patterns

### Key Benefits:

1. **Clean DAG Visualization** - Helper functions don't clutter the pipeline graph
2. **Maintainable Code** - Modular organization with clear separation of concerns
3. **Flexible Execution** - Easy switching between parallel and sequential modes
4. **Efficient Caching** - Strategic caching without serialization issues

### References

**Core Documentation:**
- [Hamilton Documentation](https://hamilton.dagworks.io/)
- [Hamilton Node Concepts](https://hamilton.dagworks.io/en/latest/concepts/node/)
- [Function Modifiers](https://hamilton.dagworks.io/en/latest/concepts/function-modifiers/)

**Best Practices:**
- [Function Naming](https://hamilton.dagworks.io/en/latest/concepts/best-practices/function-naming/)
- [Code Organization](https://hamilton.dagworks.io/en/latest/concepts/best-practices/code-organization/)

**Advanced Features:**
- [Visualization](https://hamilton.dagworks.io/en/latest/concepts/visualization/)
- [Parallel Tasks](https://hamilton.dagworks.io/en/latest/concepts/parallel-task/)
- [Caching](https://hamilton.dagworks.io/en/latest/concepts/caching/)
- [Lineage](https://hamilton.dagworks.io/en/latest/how-tos/use-hamilton-for-lineage/)

**Tools:**
- [CLI Reference](https://hamilton.dagworks.io/en/latest/how-tos/cli-reference/)
- [Hamilton Jupyter Magic](https://hamilton.dagworks.io/en/latest/how-tos/use-in-jupyter-notebook/)
- [Hamilton Jupyter Examples](https://github.com/apache/hamilton/blob/main/examples/jupyter_notebook_magic/example.ipynb)

---

*This notebook demonstrates live examples of Hamilton best practices with interactive DAG visualizations and proper imports for all cell modules.*